In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.models import ColumnDataSource

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)
#Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: float
    IsSpecialDay: bool
    VehicleType: str

In [ ]:
data = pw.demo.replay_csv("/content/parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
vehicle_weights = {"car": 1.0, "bike": 0.7, "truck": 1.5}

AttributeError: module 'pathway' has no attribute 'choose'

In [ ]:
import datetime
@pw.udf
def vehicle_type_weight(vtype: str) -> float:
    return vehicle_weights.get(vtype.lower(), 1.0)

def bool_to_float(value: bool) -> float:
    return 1.0 if value else 0.0


demand_intermediate = (
    data_with_time.with_columns(
        vweight = vehicle_type_weight(pw.this.VehicleType)
    )
    .windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        lot = pw.reducers.any(pw.this.SystemCodeNumber),
        timestamp = pw.this._pw_window_end,
        occupancy = pw.reducers.avg(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity),
        queue = pw.reducers.avg(pw.this.QueueLength),
        traffic = pw.reducers.avg(pw.this.TrafficConditionNearby),
        special = pw.reducers.max(pw.this.IsSpecialDay),
        vtype = pw.reducers.avg(pw.this.vweight),
    )
)
# Step B1: Calculate norm_demand
demand_scored = demand_intermediate.with_columns(
    norm_demand = (
        (pw.this.occupancy / pw.this.cap) +
        0.3 * pw.this.queue -
        0.2 * pw.this.traffic +
        0.4 * pw.this.special +
        0.3 * pw.this.vtype
    )
)

# Step B2: Bound norm_demand and compute price
demand_based = demand_scored.with_columns(
    norm_demand_bounded = pw.if_else(
        (pw.this.norm_demand < 0),
        0,
        pw.if_else(pw.this.norm_demand > 2.0, 2.0, pw.this.norm_demand)
    ),
    model2_price = pw.this.norm_demand_bounded * 10
)


TypeError: Pathway does not support using binary operator mul on columns of types <class 'float'>, <class 'bool'>.
It refers to the following expression:
	(0.4 * <table1>.special)
called in /tmp/ipython-input-30-3138636496.py:32
with tables:
	<table1> created in /tmp/ipython-input-30-3138636496.py:20
